# Project 2: Cleaning and Transforming Data for Analysis
## For 3 data sets chosen from the week 5 discussion forum:
### 1. Create a .csv file
### 2. Use pandas to read the file and tidy it
### 3. Perform the analysis suggested in the discussion post

## Data Set 1: Google forms (posted by Irteza Sheikh)
Although Irteza did not attach a data set to his post, I had a data set handy from a recent survey at my work
The survey was intended to find out how employees are using the software we pay for, with the intention of discontinuing usage of some, and potentially upgrading others.

In [1]:
import pandas as pd
import numpy as np

url = 'https://github.com/sarahbill33/dataacq/blob/main/Software%20Survey.csv?raw=true'
usage = pd.read_csv(url, skipinitialspace=True, index_col=0)

print(usage.head(5))

                                           Name  \
Timestamp                                         
2022/05/06 9:06:38 AM AST            Ward Betts   
2022/05/06 9:12:11 AM AST           Sean Nelson   
2022/05/06 9:18:55 AM AST   Keith Kochersperger   
2022/05/06 10:35:30 AM AST                Geoff   
2022/05/06 11:15:10 AM AST                Becky   

                           How frequently do you use the following applications? [QuickBooks]  \
Timestamp                                                                                       
2022/05/06 9:06:38 AM AST                                          Do not use                   
2022/05/06 9:12:11 AM AST                                          Do not use                   
2022/05/06 9:18:55 AM AST                                          Do not use                   
2022/05/06 10:35:30 AM AST                                              Daily                   
2022/05/06 11:15:10 AM AST                                  

In [4]:
usage.columns

Index(['Name',
       'How frequently do you use the following applications? [QuickBooks]',
       'How frequently do you use the following applications? [Fishbowl]',
       'How frequently do you use the following applications? [Asana]',
       'How frequently do you use the following applications? [SmartSheet]',
       'How frequently do you use the following applications? [Microsoft 365]',
       'How frequently do you use the following applications? [Canva]',
       'How frequently do you use the following applications? [ShipStation]',
       'How frequently do you use the following applications? [FedEx]',
       'How frequently do you use the following applications? [Amazon Seller Central]',
       'How frequently do you use the following applications? [Shopify]',
       'How frequently do you use the following applications? [SPS Commerce]',
       'How frequently do you use the following applications? [Syndigo]',
       'How frequently do you use the following applications? [WERC

Since the survey used a matrix format to gather answers, there are repeated questions with the tail end of the question showing the actual software application. To start, I will change the column heading to show the actual software name instead of the question.

In [2]:
usage.columns = ['Name','Quickbooks Usage', 'Fishbowl Usage', 'Asana Usage', 'Smartsheet Usage', 'Microsoft 365 Usage', 'Canva Usage', 'ShipStation Usage', 'FedEx Usage', 'Amazon Seller Central Usage', 'Shopify Usage', 'SPS Commerce Usage', 'Syndigo Usage', 'WERCSmart Usage', 'GS1 Usage', 'Quickbooks Permissions', 'Fishbowl Permissions', 'Asana Permissions', 'Smartsheet Permissions', 'Microsoft 365 Permissions','Canva Permissions', 'ShipStation Permissions', 'FedEx Permissions', 'Amazon Seller Central Permissions', 'Shopify Permissions', 'SPS Commerce Permissions', 'Syndigo Permissiosns', 'WERCSmart Permissions', 'GS1 Permissions', 'Comments']
print(usage.head(5))

                                           Name Quickbooks Usage  \
Timestamp                                                          
2022/05/06 9:06:38 AM AST            Ward Betts       Do not use   
2022/05/06 9:12:11 AM AST           Sean Nelson       Do not use   
2022/05/06 9:18:55 AM AST   Keith Kochersperger       Do not use   
2022/05/06 10:35:30 AM AST                Geoff            Daily   
2022/05/06 11:15:10 AM AST                Becky  Monthly or less   

                             Fishbowl Usage      Asana Usage Smartsheet Usage  \
Timestamp                                                                       
2022/05/06 9:06:38 AM AST             Daily       Do not use       Do not use   
2022/05/06 9:12:11 AM AST             Daily  Monthly or less       Do not use   
2022/05/06 9:18:55 AM AST   Monthly or less       Do not use           Weekly   
2022/05/06 10:35:30 AM AST            Daily  Monthly or less  Monthly or less   
2022/05/06 11:15:10 AM AST           

Ah, much better. Now I want to see which software is used the most.
Unfortunately, I just realized that it's going to be really tough to make those calculations without the strings being turned into numbers. So I will do that first by transforming the string ratings into number ratings:
- Daily : 3
- Weekly : 2
- Monthly or less : 1

In [3]:
usage2 = usage.replace(['Daily'], 3)
usage3 = usage2.replace(['Weekly'], 2)
usage4 = usage3.replace(['Monthly or less'], 1)
usage5 = usage4.replace(['Do not use'], 0)
usage6 = usage5.replace(['Monthly or less;Do not use'], 0)
usage7 = usage6.replace(['Daily;Do not use'], 0)

print(usage7.head(5))

                                           Name  Quickbooks Usage  \
Timestamp                                                           
2022/05/06 9:06:38 AM AST            Ward Betts                 0   
2022/05/06 9:12:11 AM AST           Sean Nelson                 0   
2022/05/06 9:18:55 AM AST   Keith Kochersperger                 0   
2022/05/06 10:35:30 AM AST                Geoff                 3   
2022/05/06 11:15:10 AM AST                Becky                 1   

                            Fishbowl Usage  Asana Usage  Smartsheet Usage  \
Timestamp                                                                   
2022/05/06 9:06:38 AM AST                3            0                 0   
2022/05/06 9:12:11 AM AST                3            1                 0   
2022/05/06 9:18:55 AM AST                1            0                 2   
2022/05/06 10:35:30 AM AST               3            1                 1   
2022/05/06 11:15:10 AM AST               3            

When performing the last round of calculations, I tried really hard to find a calculation that allowed me to replace all the values at the same time. I tried .map and .appy with a python function. I also tried using .replace with a lambda function but unfortunately none of those worked. The only successful method I found was using an iteration of new dataframes, with each one passing through a replace function.

I realize now that despite having clearer column names and number values, I need to reshape the data from wide to long.

I can probably manage my analysis either way, but it will be much easier if the data is in long format.

In [4]:
usage8 = pd.melt(usage7, id_vars='Name', value_vars=['Quickbooks Usage', 'Fishbowl Usage', 'Asana Usage', 'Smartsheet Usage', 'Microsoft 365 Usage', 'Canva Usage', 'ShipStation Usage', 'FedEx Usage', 'Amazon Seller Central Usage', 'Shopify Usage', 'SPS Commerce Usage', 'Syndigo Usage', 'WERCSmart Usage', 'GS1 Usage', 'Quickbooks Permissions', 'Fishbowl Permissions', 'Asana Permissions', 'Smartsheet Permissions', 'Microsoft 365 Permissions','Canva Permissions', 'ShipStation Permissions', 'FedEx Permissions', 'Amazon Seller Central Permissions', 'Shopify Permissions', 'SPS Commerce Permissions', 'Syndigo Permissiosns', 'WERCSmart Permissions', 'GS1 Permissions', 'Comments'])

print(usage8.head(5))

                  Name          variable value
0           Ward Betts  Quickbooks Usage     0
1          Sean Nelson  Quickbooks Usage     0
2  Keith Kochersperger  Quickbooks Usage     0
3                Geoff  Quickbooks Usage     3
4                Becky  Quickbooks Usage     1


In [5]:
values = ['Quickbooks Permissions', 'Fishbowl Permissions', 'Asana Permissions', 'Smartsheet Permissions', 'Microsoft 365 Permissions','Canva Permissions', 'ShipStation Permissions', 'FedEx Permissions', 'Amazon Seller Central Permissions', 'Shopify Permissions', 'SPS Commerce Permissions', 'Syndigo Permissiosns', 'WERCSmart Permissions', 'GS1 Permissions', 'Comments']

usage9 = usage8[usage8.variable.isin(values) == False]

print(usage9)

                    Name          variable value
0             Ward Betts  Quickbooks Usage     0
1            Sean Nelson  Quickbooks Usage     0
2    Keith Kochersperger  Quickbooks Usage     0
3                  Geoff  Quickbooks Usage     3
4                  Becky  Quickbooks Usage     1
..                   ...               ...   ...
247     Audrey Henderson         GS1 Usage     1
248     Amanda McCloskey         GS1 Usage     0
249           Sarah Bill         GS1 Usage     1
250                  Kim         GS1 Usage     0
251        Jeffrey Cates         GS1 Usage     0

[252 rows x 3 columns]


MUCH BETTER. In addition to reshaping the dataframe, I also got rid of the rows related to permissions.


Now I will see which software has the most usage.

In [6]:
usage9.groupby('variable').value.sum()

variable
Amazon Seller Central Usage    25
Asana Usage                    15
Canva Usage                    15
FedEx Usage                    23
Fishbowl Usage                 44
GS1 Usage                       6
Microsoft 365 Usage            21
Quickbooks Usage               13
SPS Commerce Usage             19
ShipStation Usage              23
Shopify Usage                  23
Smartsheet Usage               12
Syndigo Usage                   7
WERCSmart Usage                 2
Name: value, dtype: object

## After renaming, replacing, dropping, and reshaping, I can see that the software with the most usage is Amazon Seller Central. My company would definitely NOT want to drop our contract for that software. However, we might consider looking at our payment plan for WERCSmart.